In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



In [2]:
data = pd.read_csv("realtime_consumption_data.csv")

In [3]:
data.head(10)

,consumption,Tarih,hour
0,31782.78,01.12.2018,00:00
1,30016.48,01.12.2018,01:00
2,28713.42,01.12.2018,02:00
3,27838.91,01.12.2018,03:00
4,27621.64,01.12.2018,04:00
5,27740.53,01.12.2018,05:00
6,28305.31,01.12.2018,06:00
7,29061.61,01.12.2018,07:00
8,31542.21,01.12.2018,08:00
9,34223.81,01.12.2018,09:00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52632 entries, 0 to 52631
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   consumption  52632 non-null  float64
 1   Tarih        52632 non-null  object 
 2   hour         52632 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.2+ MB


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

file_path = "realtime_generation_data.csv"  
data = pd.read_csv(file_path)

print("Veri seti sütunları:", data.columns)

numeric_data = data.select_dtypes(include=['number'])

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(numeric_data)

train_size = int(len(data_scaled) * 0.7)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

def create_sequences(dataset, seq_length):
    X = []
    y = []
    for i in range(len(dataset) - seq_length):
        X.append(dataset[i:i+seq_length])
        y.append(dataset[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10  
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

model = Sequential()
model.add(LSTM(64, input_shape=(seq_length, numeric_data.shape[1])))
model.add(Dense(numeric_data.shape[1]))  

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, epochs=10, batch_size=32)

train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Train Loss: {train_loss}, Test Loss: {test_loss}')

future_data = data_scaled[-seq_length:]  
future_data = future_data.reshape((1, seq_length, numeric_data.shape[1]))
predicted_data = model.predict(future_data)

predicted_data = scaler.inverse_transform(predicted_data)
print(f'Tahmin edilen değer: {predicted_data}')



Veri seti sütunları: Index(['Tarih', 'hour', 'total'], dtype='object')


2024-12-17 17:05:43.953297: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-17 17:05:43.953321: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-17 17:05:43.953326: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-17 17:05:43.953388: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-17 17:05:43.953633: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2024-12-17 17:05:44.802468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1151/1151 [==============================] - 13s 8ms/step - loss: 0.0058
Epoch 2/10
1151/1151 [==============================] - 10s 8ms/step - loss: 9.7595e-04
Epoch 3/10
1151/1151 [==============================] - 9s 8ms/step - loss: 6.9766e-04
Epoch 4/10
1151/1151 [==============================] - 10s 8ms/step - loss: 6.2552e-04
Epoch 5/10
1151/1151 [==============================] - 10s 8ms/step - loss: 5.8657e-04
Epoch 6/10
1151/1151 [==============================] - 10s 8ms/step - loss: 5.6710e-04
Epoch 7/10
1151/1151 [==============================] - 9s 8ms/step - loss: 5.3821e-04
Epoch 8/10
1151/1151 [==============================] - 9s 8ms/step - loss: 5.0144e-04
Epoch 9/10
1151/1151 [==============================] - 10s 8ms/step - loss: 4.5825e-04
Epoch 10/10
1151/1151 [==============================] - 9s 8ms/step - loss: 4.2744e-04
Train Loss: 0.0005195781704969704, Test Loss: 0.0008430284215137362
1/1 [==============================] - 2s 2s/step
Tahmin edilen değer: